#### Get text from board screenshot

In [1]:
import easyocr
from PIL import Image
import numpy as np

from spellchecker import SpellChecker

#import Libraries
from sentence_transformers import SentenceTransformer
import numpy as np
from unidecode import unidecode

c:\Users\vijay\vscode  projects\Codenames optimizer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Take screenshot
img = Image.open("sc_8.png")
img_arr = np.array(img)

In [3]:
# OCR
reader = easyocr.Reader(['en'])
results = reader.readtext(img_arr)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\vijay\vscode  projects\Codenames optimizer\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [4]:
#fix the text from easyocr
board_0_fix = [i.replace("0","O") for _,i,_ in results]

spell = SpellChecker()

board = [spell.correction(w).upper() for w in board_0_fix]
board

['UNIFORM',
 'EGYPT',
 'BOOM',
 'ANTARCTICA',
 'DANCE',
 'WEB',
 'AUSTRALIA',
 'HITCHHIKER',
 'CIRCLE',
 'BLACKBERRY',
 'BAND',
 'SUIT',
 'CLOAK',
 'CZECH',
 'GLASS',
 'ROCK',
 'HURRICANE',
 'HONEY',
 'DECK',
 'HARBOR',
 'PISTOL',
 'VIOLIN',
 'MOLE',
 'PASS',
 'CALF']

In [5]:
#test getting word embeddings
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [6]:
#helpful functions
def norm(x):
    x = x / (np.linalg.norm(x, axis=1, keepdims=True) + 1e-9)
    return x
def embed(words):
    w = [unidecode(w.lower().replace("_"," ").strip()) for w in words]
    v = model.encode(w, normalize_embeddings=True)
    return w, v

In [7]:
#get board word embeddings
_,board_embeddings = embed(board)


In [8]:
def get_choices_from_hint(hint,k=25):
    hint_embedding = model.encode(hint, normalize_embeddings=True).reshape(1,-1)
    sims = (board_embeddings @ hint_embedding.T).ravel()
    return [(board[i], float(sims[i])) for i in np.argsort(-sims)][:k]

#### Use the Hint and Number to get the words to choose

In [16]:
get_choices_from_hint("scrolls",6)

[('DECK', 0.4809088706970215),
 ('CLOAK', 0.40676310658454895),
 ('WEB', 0.3379850387573242),
 ('SUIT', 0.31046438217163086),
 ('BAND', 0.30157557129859924),
 ('DANCE', 0.2906075716018677)]